## EPSG:5187 기준을 위도 경도 로 수정하기

In [ ]:
import pandas as pd
from pyproj import Transformer

# CSV 파일 로드
df = pd.read_csv("raster_export.csv")

# 중심점(x, y) 계산 (EPSG:5187 기준)
df['x'] = (df['left'] + df['right']) / 2
df['y'] = (df['top'] + df['bottom']) / 2

# 좌표 변환기 정의: EPSG:5187 → EPSG:4326 (WGS84)
transformer = Transformer.from_crs("EPSG:5187", "EPSG:4326", always_xy=True)

# 위도/경도 변환 적용
df['longitude'], df['latitude'] = transformer.transform(df['x'].values, df['y'].values)

# 결과 확인
print(df[['id', 'latitude', 'longitude']].head())


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# CSV 읽기
df = pd.read_csv("input.csv")

# 중심 좌표 계산
df['x_center'] = (df['left'] + df['right']) / 2
df['y_center'] = (df['top'] + df['bottom']) / 2

# 중심 좌표를 Point 객체로 변환
geometry = [Point(xy) for xy in zip(df['x_center'], df['y_center'])]

# ✅ EPSG:5186 (TM 중부 원점) 지정
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:5186")

# 위경도 좌표계(WGS84)로 변환
gdf = gdf.to_crs("EPSG:4326")

# 위도/경도 열 추가
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y

# 불필요한 열 삭제
gdf = gdf.drop(columns=["left", "right", "top", "bottom", "x_center", "y_center", "geometry"])

# CSV로 저장
gdf.to_csv("output_converted.csv", index=False)
